## Workflow Multi-Agent Pattern

#### En mis pruebas, en algunas ocasiones el Workflow Tool falló. Workflow se encuentra en [active development](https://github.com/strands-agents/tools/commits/main/src/strands_tools/workflow.py).

### Mi Recomendación personal, sobre multi-agent en Strands Agents:
➡️ Dependiendo de tu caso de uso, comienza con Agents as Tools, Graph o Swarm

```
# Ultimas pruebas hechas con versión:
strands-agents-tools-0.2.14
```

- Otro motivo de fallo son los RPM que tengamos en nuestra quota.

Documentación: Ver más en [Strands Agents Graphs](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/multi-agent/workflow/)

---

In [ ]:
import logging

# Enable debug logs and print them to stderr
logging.getLogger("strands.multiagent").setLevel(logging.DEBUG)
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)

> En un flujo de trabajo secuencial, los agentes procesan tareas en un orden definido, donde la salida de cada agente se convierte en la entrada del siguiente:

In [ ]:
from strands import Agent

# Create specialized agents
researcher = Agent(system_prompt="You are a research specialist. Find key information and return a quick summary.", callback_handler=None)
analyst = Agent(system_prompt="You analyze research data and extract summarized insights.", callback_handler=None)
writer = Agent(system_prompt="You create polished and summarized reports, based on analysis.")

# Sequential workflow processing
def process_workflow(topic):
    # Step 1: Research
    research_results = researcher(f"Research the latest developments in {topic}")

    # Step 2: Analysis
    analysis = analyst(f"Analyze these research findings: {research_results}")

    # Step 3: Report writing
    final_report = writer(f"Create a report in Spanish, based on this analysis: {analysis}")

    return final_report

In [ ]:
from IPython.display import display, Markdown

# Call Workflow
response = process_workflow("Seguridad y cifrado en mensajería de Whatsapp")

# Display Results
content = response.message['content'][0]['text']
display(Markdown(content))

In [ ]:
# Muestra Tokens
response.metrics.accumulated_usage

---

### Usando el Tool de Workflow

> Si quieren ver el código de esta Tool, recordando que Strands Agents es open source, [aquí lo tienen](https://github.com/strands-agents/tools/blob/main/src/strands_tools/workflow.py)

In [ ]:
from strands import Agent
from strands_tools import workflow

# Create an agent with workflow capability
agent = Agent(tools=[workflow])

# Create a multi-agent workflow
agent.tool.workflow(
    action="create",
    workflow_id="whatsapp_security_analysis",
    tasks=[
        {
            "task_id": "research",
            "description": "Research the latest developments in WhatsApp security and encryption",
            "system_prompt": "You are a research specialist. Find key information and return a brief summary.",
            "priority": 5
        },
        {
            "task_id": "analysis",
            "description": "Analyze the research findings and extract summarized insights",
            "dependencies": ["research"],
            "system_prompt": "You analyze research data and extract summarized insights.",
            "priority": 3
        },
        {
            "task_id": "report_writing",
            "description": "Create a polished and summarized report in Spanish based on the analysis",
            "dependencies": ["analysis"],
            "system_prompt": "You create polished and summarized reports, based on analysis.",
            "priority": 2
        }
    ]
)

# Execute workflow with your input
response = agent.tool.workflow(
                action="start",
                workflow_id="whatsapp_security_analysis",
                input="Cuáles son los últimos avances en Seguridad y cifrado en mensajería de Whatsapp"
            )

response['content'][0]['text']

In [ ]:
# Metrics
status = agent.tool.workflow(action="status", workflow_id="whatsapp_security_analysis")

# Display Results
response = status['content'][0]['text']
print(response)

### Borra Workflow

In [ ]:
# Delete
status = agent.tool.workflow(action="delete", workflow_id="whatsapp_security_analysis")

# Display Results
response = status['content'][0]['text']
print(response)